The genetic algorithm is a computer approximation of how evolution performs research, which involves making changes to the parent genomes in their offspring and thus producing new individuals with different abilities. 

In this file, we will build a genetic algorithm with Python by solving a real-time case study.

### How does a Genetic Algorithm work?

Like other mathematical models such as neurons, a genetic algorithm tries to ignore everything except the important parts that we need to understand what evolution does. From this principle, the elements we need to model a simple genetic algorithm inside a computer and solve problems with it are:

* a method of representing problems in the form of chromosomes.
* a way to calculate the adequacy of a solution.
* a selection method for choosing parents.
* a way to generate offspring by raising parents.

We now have understood what a genetic algorithm is. Now let’s head over to a case study to get into a situation where we can build our genetic algorithm.

### Case Study Based on a Genetic Algorithm:

We have **[X] MP3 files** on our computer hard drive. Unfortunately, the hard drive started making noise and we decide that we had better save the **MP3s**. Likewise, unfortunately, we can only **burn CDs, not DVDs**, on our computer. We need to **minimize the number of CDs we use, so we decide to design a genetic algorithm to choose which MP3s to put on each CD to fill each CD as completely as possible**. Design a genetic algorithm to solve the problem. 

We will need to think about how we would encode the entries, the appropriate genetic operators, and how we would make the genetic algorithm handle the fact that we have multiple CDs, not just one CD.

The first thing we need to look at is how to represent the problem in the form of chromosomes. As we are having **[X] number of files**. We should build a binary chromosome where 
* `1` means a particular MP3 is taken from the chromosome while 
* `0` means it is not. 

The position of this bit will indicate which MP3 we will start with.

### Building a Genetic Algorithm

In [1]:
import numpy as np
import random

def generateParents(size):
    parents = np.array(random.randint(0, 2**size - 1))
    for i in range(1, population):
        parents = np.append(parents, random.randint(0, 2**size - 1))
    return parents

* `GenerateParents` is our parent generation function. Since this is the easiest to do, we’ll do it first. 
* The `population` will contain the total number of descendants that each generation will have and for the first generation, this will be the total number of randomly generated parents. 
* `size` contains the current total number of MP3s being processed.

In [2]:
def totalSize(data, size):
    s = 0
    for i in range(0, size-1):
        if(data & (1 << i) > 0):
            s += mp3s[i]
    return s

* `totalSize` is a simple function that measures the total space used by all selected MP3s in that particular chromosome (data). 
* `size` contains the current total number of MP3s being processed. For each bit of the chromosome, we check if there is one and if so, we increment `s` by the size of the MP3 file corresponding to that bit.

In [3]:
def reduceSize(rec, size):
    while totalSize(rec, size) > 700:
        index = random.randint(0, size - 1)
        if(rec & (1 << index) > 0):
            rec = rec ^ (1 << index)
    return rec

* `reduceSize` is the function we use to randomly mutate the chromosome in a way that reduces the total size of the MP3 files described by that chromosome to fit on a CD. 
* So as long as the total chromosome **size exceeds 700**, we choose a random bit and if it is `1`, we change it to `0`.

In [4]:
def mutate(rec, size):
    index = random.randint(0, size - 1)
    rec = rec ^ (1 << index)
    return rec

* `mutate` is our mutating function. Mutations happen in real life when new descendants are created, usually, mutations would have a mutation rate, but we are omitting that in this file for simplicity. 
* Our mutation function selects a bit at random and changes its value.

In [5]:
def fixChromosomes(data, size, population):
    datasize = data.shape[0]
    fitness = np.zeros((datasize,1), dtype=int)
    for i in range(0, datasize):
        rec = data[i]
        if(totalSize(rec, size) > 700):
            rec = reduceSize(rec, size)
            data[i] = rec
        fitness[i] = -1* totalSize(data[i], size)
    data = np.transpose(np.array([data]))
    generation = np.concatenate((data, fitness), axis=1)
    generation = generation[generation[:population, 1].argsort()]
    return generation

* `fixChromosomes` will take the data and apply it to the `reduceSize` function to the chromosomes as needed. 
* It also applies the `fitness` function and adds it to the **generation data** so that each chromosome has a corresponding fitness. This is also sorted so that those above have the best physical shape.

In [6]:
def crossover(mom, dad, size):
    index = random.randint(1, size - 1)
    mom1 = mom & (2**index -1)
    mom2 = mom & ((2**(size-index) -1) << index)
    dad1 = dad & (2**index -1)
    dad2 = dad & ((2**(size-index) -1) << index)
    return mutate(mom1|dad2, size), mutate(dad1|mom2, size)

* The `crossover` function takes 2 parents and performs a random cross between their chromosomes. 
* It chooses a random index `i` and divides mum and dad on the `i-th` index then crosses combines them both to generate 2 children. These children are then mutated by the mutation function.

In [7]:
def newGeneration(generation, size):
    top4 = generation[:4, 0]
    newGen = generation[:2,0]
    for i in range(0, 4):
        for j in range(0, 4):
            if(i != j):
                c1, c2 = crossover(top4[i], top4[j], size)
                newGen = np.append(newGen, c1)
                newGen = np.append(newGen, c2)
                #print(newGen)
    return newGen

* The `newGeneration` function takes the current generation and produces the next generation. This is done by taking the 4 best parents in terms of fitness and crossing each pair of them to generate new offspring.

However, due to the particular configuration of the problem, we are very likely to have a near-optimal solution straight out of the first generation due to the randomness and our reduction feature. For this reason, we can add the top 2 parents to the new generation to ensure that we don’t lose the optimal results created in previous generations.

### Train Our Algorithm

The train function is where everything comes together. **mp3Cnt** contains the total number of MP3 files that have not yet been classified on a CD. Every time we process the generation cycle we find a chromosome that best matches the current CD that we are working on.

In [8]:
def train(mp3Cnt, mp3s, population, generationsPerCD):
    curCD = 1
    combinedSizes = totalSize(2**mp3Cnt-1, mp3Cnt)
    doneSizes = 0.0
    while(True):
        if(mp3Cnt == 0):
            break
        parents = generateParents(mp3Cnt)
        generation = fixChromosomes(parents, mp3Cnt, population)
        ng = generation
        for i in range(generationsPerCD):
            ng = newGeneration(ng, mp3Cnt)
            ng = fixChromosomes(ng, mp3Cnt, population)
        allFileSize = totalSize(2**mp3Cnt-1, mp3Cnt)
        cdContents = ng[0,0]
        if(allFileSize < 700):
            cdContents = 2**mp3Cnt -1
        currentBestCDSize = totalSize(cdContents, mp3Cnt)
        if(currentBestCDSize >= 699 or allFileSize < 700):
            indexesToRemove = []
            for i in range(0, mp3Cnt):
                if(cdContents & (1 << i) > 0):
                    indexesToRemove.append(i)
            indexesToRemove = list(reversed(indexesToRemove))
            doneSizes += currentBestCDSize
            print("CD"+ str(curCD) + ": MP3 Count:" + str(len(indexesToRemove)) + " Size: " + str(currentBestCDSize))
            mp3Cnt = mp3Cnt - len(indexesToRemove)
            for i in range(len(indexesToRemove)):
                mp3s = np.delete(mp3s, indexesToRemove[i])
            curCD = curCD + 1
        else:
            continue